# Experimentos

### Configuração

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

Aqui está a Aplicação RAG com a qual temos trabalhado ao longo deste curso

In [ ]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langsmith import traceable
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from typing import List
import nest_asyncio

# TODO: Configure este modelo!
MODEL_NAME = "gemini-1.5-flash"
MODEL_PROVIDER = "google"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Se o armazenamento vetorial existir, então carregá-lo
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Caso contrário, indexar documentos LangSmith e criar novo armazenamento vetorial
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Retorna documentos buscados de um armazenamento vetorial baseado na pergunta do usuário
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Chama `call_openai` para gerar uma resposta do modelo após formatar entradas
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_gemini(messages)

"""
call_gemini
- Retorna a saída de conclusão de chat do Google Gemini
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_gemini(messages: List[dict]) -> str:
    return call_gemini_chat(MODEL_NAME, messages, 0.0)

"""
langsmith_rag
- Chama `retrieve_documents` para buscar documentos
- Chama `generate_response` para gerar uma resposta baseada nos documentos buscados
- Retorna a resposta do modelo
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

### Experimento

Aqui está um trecho de código que deve parecer similar ao que você vê no código inicial!

Existem alguns componentes importantes aqui.

1. Definimos um Avaliador
2. Canalizamos nossos exemplos de dataset (dict) para o formato de entrada que nossa função `langsmith_rag` aceita (str) usando uma função de destino

In [ ]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langsmith import traceable
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat
from typing import List
import nest_asyncio

# TODO: Configure este modelo!
MODEL_NAME = "gemini-1.5-flash"
MODEL_PROVIDER = "google"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """Você é um assistente para tarefas de perguntas e respostas.
Use as seguintes partes do contexto recuperado para responder à pergunta mais recente na conversa.
Se você não souber a resposta, apenas diga que não sabe.
Use no máximo três frases e mantenha a resposta concisa.
"""

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Se o armazenamento vetorial existir, então carregá-lo
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Caso contrário, indexar documentos LangSmith e criar novo armazenamento vetorial
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Retorna documentos buscados de um armazenamento vetorial baseado na pergunta do usuário
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Chama `call_gemini` para gerar uma resposta do modelo após formatar entradas
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_gemini(messages)

"""
call_gemini
- Retorna a saída de conclusão de chat do Google Gemini
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_gemini(messages: List[dict]) -> str:
    return call_gemini_chat(MODEL_NAME, messages, 0.0)

"""
langsmith_rag
- Chama `retrieve_documents` para buscar documentos
- Chama `generate_response` para gerar uma resposta baseada nos documentos buscados
- Retorna a resposta do modelo
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

### Modificando sua Aplicação

Agora, vamos mudar nosso modelo para gemini-1.5-pro e ver como ele se desempenha!

Faça essa mudança e depois execute este trecho de código!

In [ ]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gemini-1.5-pro"
)

### Executando sobre Diferentes Pedaços de Dados

##### Versão do Dataset

Você pode executar um experimento em uma versão específica de um dataset no sdk usando o parâmetro `as_of` em `list_examples`

Vamos tentar executar apenas em nosso dataset inicial.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # Usamos as_of para especificar uma versão
    evaluators=[is_concise_enough],
    experiment_prefix="versão do dataset inicial"
)

##### Divisão do Dataset

Você pode executar um experimento em uma divisão específica do seu dataset, vamos tentar executar na divisão Crucial Examples.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # Passamos uma lista de Divisões
    evaluators=[is_concise_enough],
    experiment_prefix="divisão Crucial Examples"
)

##### Pontos de Dados Específicos

Você pode especificar pontos de dados individuais para executar um experimento também

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # Passamos uma lista específica de example_ids
            # TODO: Você precisará colar seus próprios example ids para isso funcionar!
            "",
            ""
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="dois example ids específicos"
)

### Outros Parâmetros

##### Repetições

Você pode executar um experimento várias vezes para garantir que tenha resultados consistentes

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="duas repetições",
    num_repetitions=2   # Este campo tem padrão 1
)

##### Concorrência
Você também pode iniciar threads concorrentes de execução para fazer seus experimentos terminarem mais rápido!

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concorrência",
    max_concurrency=3,  # Este campo tem padrão None, então isso é uma melhoria!
)

##### Metadados

Você pode (e deve) adicionar metadados aos seus experimentos, para torná-los mais fáceis de encontrar na UI

In [ ]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadados adicionados",
    metadata={  # Podemos passar metadados personalizados para o experimento, como o nome do modelo
        "model_name": MODEL_NAME
    }
)